## First estimate

Test to check the accuracy of an estimate based on the time of arrival only

In [1]:
from pd4ml import Airshower
from matplotlib import pyplot as plt 
THINNING = 1 # Thinning factor: takes one sample among a fixed number

# Load data
x,y = Airshower.load_data("train")

# Takes only the time of arrival over the thinned dataset
toa_train = x['features'][0][::THINNING,:,-1].reshape((-1,9,9,1))
y = y[::THINNING]
del x # Remove unused data from heap

x_test, y_test = Airshower.load_data("test")
toa_test =  x_test['features'][0][::THINNING,:,-1].reshape((-1,9,9,1))
y_test = y_test[::THINNING]
del x_test

Since the data manifest some sort of redundance (angle, height, dispersion angle and average speed give the t.o.a. matrix, 4 parameters to 81 observations)  an encoder network is tested

In [2]:
from keras import layers
from keras.models import Model
from keras.utils.vis_utils import plot_model

input = layers.Input(shape=(9, 9, 1))
x = layers.Flatten()(input)
x = layers.Dense(81, activation="relu")(x)
x = layers.Dense(27, activation="relu")(x)
x = layers.Dense(9, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)
x = layers.Dense(1)(x)

retino = Model(input, x)
retino.compile(optimizer="adam", loss="mean_squared_error")
# plot_model(retino, show_shapes=True, show_layer_names=True)
retino.summary()

2022-06-22 15:43:45.336967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-22 15:43:45.337023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 1)]         0         
                                                                 
 flatten (Flatten)           (None, 81)                0         
                                                                 
 dense (Dense)               (None, 81)                6642      
                                                                 
 dense_1 (Dense)             (None, 27)                2214      
                                                                 
 dense_2 (Dense)             (None, 9)                 252       
                                                                 
 dense_3 (Dense)             (None, 4)                 40        
                                                                 
 dense_4 (Dense)             (None, 4)                 20    

2022-06-22 15:43:49.244628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-22 15:43:49.244684: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-22 15:43:49.244723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (djanloo-ratchet): /proc/driver/nvidia/version does not exist
2022-06-22 15:43:49.245490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
retino.fit(
    x=toa_train,
    y=y,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(toa_test, y_test),
)


Epoch 1/50
547/547 [==============================] - 3s 5ms/step - loss: 174831.1562 - val_loss: 5307.6182
Epoch 2/50
547/547 [==============================] - 2s 4ms/step - loss: 3588.7498 - val_loss: 3115.1138
Epoch 3/50
547/547 [==============================] - 2s 4ms/step - loss: 3003.8486 - val_loss: 2968.6387
Epoch 4/50
547/547 [==============================] - 2s 4ms/step - loss: 2913.8499 - val_loss: 2965.5520
Epoch 5/50
547/547 [==============================] - 2s 4ms/step - loss: 2881.6929 - val_loss: 2912.8010
Epoch 6/50
547/547 [==============================] - 2s 4ms/step - loss: 2858.9333 - val_loss: 2906.5847
Epoch 7/50
547/547 [==============================] - 2s 4ms/step - loss: 2835.3533 - val_loss: 2876.5344
Epoch 8/50
547/547 [==============================] - 2s 4ms/step - loss: 2820.8433 - val_loss: 2932.5625
Epoch 9/50
547/547 [==============================] - 2s 4ms/step - loss: 2799.8945 - val_loss: 2850.3188
Epoch 10/50
547/547 [=======================

In [4]:
import numpy as np
errors = np.std( retino.predict(toa_test).squeeze() - y_test )/y_test*100
print(np.mean(errors))

938/938 [==============================] - 2s 2ms/step
6.804388460265896
